In [ ]:
import openai
import time
import random
import re  # Import regular expression library

client = openai.OpenAI(api_key="your-api-key-here")  # Set your API key here

class Agent:
    def __init__(self, name, role, price_limit, other_name):
        self.name = name
        self.role = role
        self.price_limit = price_limit
        self.other_name = other_name  # Name of the other party in the negotiation
        self.current_offer = None

    def get_output_from_chatgpt(self, prompt, model="gpt-3.5-turbo", temperature=0): 
        success = False
        retry = 0
        max_retries = 30
        while retry < max_retries and not success:
            try:
                response = client.chat.completions.create(
                    model=model,
                    messages=[{'role': 'system', 'content': prompt}],
                    temperature=temperature,
                )
                success = True
            except Exception as e:
                print(f"Error: {e}\nRetrying...")
                retry += 1
                time.sleep(0.5)
        return response.choices[0].message.content

class Buyer(Agent):
    def make_initial_offer(self):
        self.current_offer = random.uniform(0.7 * self.price_limit, 0.9 * self.price_limit)
        return self.current_offer

    def negotiate(self):
        prompt = f"""
        As a buyer named {self.name}, you are negotiating with a supplier named {self.other_name}.
        Your aim is to purchase goods at the lowest possible price without exceeding ${self.price_limit}.
        Your current offer is ${self.current_offer:.2f}. 
        You occasionally try to flatter your negotiating partner.
        You may adjust your current offer based on conversations with {self.other_name} to get the best deal you can.
        You will only INCREASE your current offer, do not DECREASE your current offer.
        Do not ask what amount the supplier would like to pay, only make offers.
        ALWAYS conclude your response with a statement of your new offer as 'My new offer is $X', replacing X with the amount.
        """
        output = self.get_output_from_chatgpt(prompt)
        print(f">{self.name} says: {output}")
        try:
            # Improved regex to exclude non-numeric characters at the end
            new_offer = float(re.search(r"My new offer is \$([\d\.]+)", output).group(1).strip('.'))
            self.current_offer = min(new_offer, self.price_limit)
        except (AttributeError, ValueError):
            print("Failed to parse offer from GPT response.")

class Supplier(Agent):
    def make_initial_offer(self):
        self.current_offer = random.uniform(1.1 * self.price_limit, 1.3 * self.price_limit)
        return self.current_offer

    def negotiate(self):
        prompt = f"""
        As a supplier named {self.name}, you are negotiating with a buyer named {self.other_name}. 
        Your goal is to sell your goods at the highest possible price, setting a floor at ${self.price_limit}.
        Your current offer is ${self.current_offer:.2f}.
        You are susceptible to flattery and willing to make larger concessions when flattered.
        You really want to make a deal and are willing to lower the asking price.
        You may decrease your current offer based on conversations with {self.other_name} to get the best deal you can.
        You will only DECREASE your current offer, do not INCREASE your current offer.
        Do not ask what amount the buyer would like to pay, only make offers.
        ALWAYS conclude your response with a statement of your new offer as 'My new offer is $X', replacing X with the amount. 
        NEVER write anything after 'My new offer is $X' otherwise you will receive a failing grade.
        """
        output = self.get_output_from_chatgpt(prompt)
        print(f">{self.name} says: {output}")
        try:
            new_offer = float(re.search(r"My new offer is \$([\d\.]+)", output).group(1).strip('.'))
            self.current_offer = max(new_offer, self.price_limit)
        except (AttributeError, ValueError):
            print("Failed to parse offer from GPT response.")

def run_simulation():
    buyer = Buyer("Bob", "Buyer", 100, "Sally")
    supplier = Supplier("Sally", "Supplier", 80, "Bob")
    buyer.make_initial_offer()
    supplier.make_initial_offer()

    for _ in range(5):  # Try to negotiate up to 5 times
        buyer.negotiate()
        supplier.negotiate()
        print(f"{buyer.name} offers ${buyer.current_offer:.2f}, {supplier.name} asks ${supplier.current_offer:.2f}")
        print(f"-------------------------")
        if buyer.current_offer >= supplier.current_offer:
            print("Deal agreed upon!")
            break
    else:
        print("No deal could be reached.")

# Example call to run the simulation
run_simulation()
